In [ ]:
w#Transcript Scraping



In [ ]:
#Transcript Scraping Cell

import requests, time, re, os, csv
from pathlib import Path
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm

BASE    = "https://www.asapsports.com"
YEARS   = [2014, 2015, 2016, 2017, 2018, 2019]
OUTPUT  = "mlb_postseason_transcripts_2014_2019.csv"
OUTPATH = Path(OUTPUT)
DELAY   = 0.3
MIN_WORDS = 50

# Include postseason; exclude non-MLB
INCLUDE_PATTERNS = [
    r"\bALDS\b", r"\bNLDS\b", r"\bALCS\b", r"\bNLCS\b",
    r"\bDIVISION\s+SERIES\b",
    r"\bLEAGUE\s+CHAMPIONSHIP\s+SERIES\b",
    r"\bCHAMPIONSHIP\s+SERIES\b",
    r"\bWORLD\s+SERIES\b",
]
EXCLUDE_PATTERNS = [
    r"\bCOLLEGE\b", r"\bNCAA\b", r"\bCOLLEGIATE\b",
    r"\bWORLD\s+BASEBALL\s+CLASSIC\b", r"\bWBC\b",
    r"\bLITTLE\s+LEAGUE\b", r"\bALL[-\s]?STAR\b", r"\bWINTER\s+MEETINGS\b",
    r"\bWILD[-\s]?CARD\b", r"\bWILDCARD\b",
]
INCLUDE_RE = re.compile("|".join(INCLUDE_PATTERNS), re.IGNORECASE)
EXCLUDE_RE = re.compile("|".join(EXCLUDE_PATTERNS), re.IGNORECASE)
QA_MARKERS_RE = re.compile(r'(?:^|\s)(Q[\.\:\-\u2013\u2014])', re.IGNORECASE)

def is_mlb_postseason(txt):
    if not isinstance(txt, str): return False
    if EXCLUDE_RE.search(txt):  return False
    return bool(INCLUDE_RE.search(txt))

def norm_url(href):
    if href.startswith("http://"): href = "https://" + href[7:]
    if href.startswith("https://"): return href
    if href.startswith("/"): return BASE + href
    return BASE + "/" + href.lstrip("/")

def safe_get(sess, url, retries=2, sleep=1, timeout=18):
    for _ in range(retries):
        try:
            r = sess.get(url, timeout=timeout)
            if r.status_code == 200 and r.text:
                return r.text
        except Exception:
            pass
        time.sleep(sleep)
    return None

def clean_space(s): return re.sub(r"\s+", " ", s or "").strip()

def extract_transcript(soup):
    # 1) paragraphs
    ptxt = clean_space(" ".join(p.get_text(" ", strip=True) for p in soup.find_all("p")))
    if len(ptxt.split()) >= MIN_WORDS: return ptxt
    # 2) broader containers
    collected = []
    for tag in ("div","font","span","td"):
        for el in soup.find_all(tag):
            t = el.get_text(" ", strip=True)
            if t: collected.append(t)
    broad = clean_space(" ".join(collected))
    m = QA_MARKERS_RE.search(broad)
    if m: broad = clean_space(broad[m.start():])
    if len(broad.split()) >= MIN_WORDS: return broad
    # 3) full body fallback
    body = clean_space(soup.get_text(" ", strip=True))
    m2 = QA_MARKERS_RE.search(body)
    if m2: body = clean_space(body[m2.start():])
    return body

def gather_interviews_on_page(sess, url):
    html = safe_get(sess, url)
    if not html: return [], ("","")
    soup = BeautifulSoup(html, "html.parser")
    title = soup.find("title").get_text(" ", strip=True) if soup.find("title") else ""
    body  = soup.get_text(" ", strip=True)
    links = [norm_url(a.get("href")) for a in soup.select('a[href*="show_interview.php?id="]') if a.get("href")]
    if "show_interview.php?id=" in url: links.insert(0, url)
    # dedupe
    seen=set(); links=[u for u in links if not (u in seen or seen.add(u))]
    return links, (title, body)

# --- Streaming writer: create file early, maintain seen set for dedupe
if not OUTPATH.exists():
    with OUTPATH.open("w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["year","interview_url","title","word_count","transcript"])

seen_urls = set()
if OUTPATH.stat().st_size > 0:
    try:
        existing = pd.read_csv(OUTPATH, usecols=["interview_url"])
        seen_urls = set(existing["interview_url"].dropna().tolist())
    except Exception:
        pass

def flush_rows(rows):
    if not rows: return
    with OUTPATH.open("a", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        for r in rows:
            w.writerow([r["year"], r["interview_url"], r["title"], r["word_count"], r["transcript"]])

session = requests.Session()
session.headers.update({"User-Agent": "Mozilla/5.0 (ASAP-MLB-Stream/1.0)"})

buffer = []
per_year_added = {}

for year in tqdm(YEARS, desc="Years"):
    start_added = 0

    y_url = f"{BASE}/show_year.php?category=2&year={year}"
    y_html = safe_get(session, y_url)
    if not y_html:
        print(f"⚠️ No year page for {year}")
        continue
    y_soup = BeautifulSoup(y_html, "html.parser")

    # gather series and day pages (lenient at this stage)
    series_pages = [norm_url(a["href"]) for a in y_soup.select('a[href*="show_events.php?event_id="]')]
    day_pages    = [norm_url(a["href"]) for a in y_soup.select('a[href*="show_event.php?event_id="]')]
    # dedupe
    def dedupe(seq):
        seen=set(); out=[]
        for x in seq:
            if x not in seen:
                out.append(x); seen.add(x)
        return out
    series_pages = dedupe(series_pages)
    day_pages    = dedupe(day_pages)

    print(f"{year}: {len(series_pages)} series pages, {len(day_pages)} day pages")

    # targets to crawl
    targets = set(day_pages)
    for sp in series_pages:
        sp_html = safe_get(session, sp)
        if not sp_html: continue
        sp_soup = BeautifulSoup(sp_html, "html.parser")
        # harvest day pages + any direct interviews
        for a in sp_soup.select('a[href*="show_event.php?event_id="]'):
            targets.add(norm_url(a.get("href")))
        for a in sp_soup.select('a[href*="show_interview.php?id="]'):
            targets.add(norm_url(a.get("href")))

    targets = list(targets)
    print(f"{year}: crawling {len(targets)} day/interview pages...")

    # scrape interviews; stream to disk every 20 captures
    for targ in tqdm(targets, leave=False, desc=f"{year} pages"):
        links, (t_title, t_body) = gather_interviews_on_page(session, targ)
        # loosen at container level; enforce strictly at interview level below
        for iu in links:
            if iu in seen_urls:  # skip already saved
                continue
            ih = safe_get(session, iu)
            if not ih:
                time.sleep(DELAY);
                continue
            isoup = BeautifulSoup(ih, "html.parser")
            i_title = isoup.find("title").get_text(" ", strip=True) if isoup.find("title") else ""
            i_body  = isoup.get_text(" ", strip=True)

            # strict postseason check (title/body/url)
            if not (is_mlb_postseason(i_title) or is_mlb_postseason(i_body) or is_mlb_postseason(iu)):
                continue

            transcript = extract_transcript(isoup)
            wc = len(transcript.split())
            if wc < MIN_WORDS:
                continue

            row = {"year": year, "interview_url": iu, "title": i_title, "word_count": wc, "transcript": transcript}
            buffer.append(row)
            seen_urls.add(iu)
            start_added += 1

            if len(buffer) >= 20:
                flush_rows(buffer)
                buffer.clear()
                # quick visibility:
                print(f"  ↳ streamed {start_added} for {year} (file size now {OUTPATH.stat().st_size/1024:.1f} KB)")

            time.sleep(DELAY)

    # flush any remaining from this year
    if buffer:
        flush_rows(buffer)
        buffer.clear()

    per_year_added[year] = start_added
    print(f"✓ {year}: added {start_added} (running total rows: {sum(per_year_added.values())})  →  {OUTPUT}")

print("\nPer-year added:", per_year_added)
print("Done.")


Years:   0%|          | 0/6 [00:00<?, ?it/s]

2014: 17 series pages, 0 day pages
2014: crawling 88 day/interview pages...


2014 pages:   0%|          | 0/88 [00:00<?, ?it/s]

  ↳ streamed 20 for 2014 (file size now 324.1 KB)
  ↳ streamed 40 for 2014 (file size now 562.3 KB)
  ↳ streamed 60 for 2014 (file size now 814.7 KB)
  ↳ streamed 80 for 2014 (file size now 1131.4 KB)
  ↳ streamed 100 for 2014 (file size now 1421.6 KB)
  ↳ streamed 120 for 2014 (file size now 1682.7 KB)
  ↳ streamed 140 for 2014 (file size now 1957.3 KB)
  ↳ streamed 160 for 2014 (file size now 2313.1 KB)
  ↳ streamed 180 for 2014 (file size now 2647.5 KB)
  ↳ streamed 200 for 2014 (file size now 2932.2 KB)
  ↳ streamed 220 for 2014 (file size now 3277.4 KB)
  ↳ streamed 240 for 2014 (file size now 3587.7 KB)
  ↳ streamed 260 for 2014 (file size now 4002.4 KB)
  ↳ streamed 280 for 2014 (file size now 4305.3 KB)
✓ 2014: added 290 (running total rows: 290)  →  mlb_postseason_transcripts_2014_2019.csv
2015: 19 series pages, 0 day pages
2015: crawling 101 day/interview pages...


2015 pages:   0%|          | 0/101 [00:00<?, ?it/s]

...

  ↳ streamed 20 for 2017 (file size now 14830.4 KB)
  ↳ streamed 40 for 2017 (file size now 15124.1 KB)
  ↳ streamed 60 for 2017 (file size now 15428.0 KB)
  ↳ streamed 80 for 2017 (file size now 15721.8 KB)
  ↳ streamed 100 for 2017 (file size now 15977.0 KB)
  ↳ streamed 120 for 2017 (file size now 16269.6 KB)
  ↳ streamed 140 for 2017 (file size now 16576.9 KB)
  ↳ streamed 160 for 2017 (file size now 16839.5 KB)
  ↳ streamed 180 for 2017 (file size now 17143.8 KB)
  ↳ streamed 200 for 2017 (file size now 17458.5 KB)
  ↳ streamed 220 for 2017 (file size now 17802.7 KB)
  ↳ streamed 240 for 2017 (file size now 18164.2 KB)
  ↳ streamed 260 for 2017 (file size now 18476.5 KB)
  ↳ streamed 280 for 2017 (file size now 18826.1 KB)
  ↳ streamed 300 for 2017 (file size now 19106.8 KB)
  ↳ streamed 320 for 2017 (file size now 19431.4 KB)
  ↳ streamed 340 for 2017 (file size now 19687.0 KB)
✓ 2017: added 347 (running total rows: 1288)  →  mlb_postseason_transcripts_2014_2019.csv
2018: 16 seri

2018 pages:   0%|          | 0/86 [00:00<?, ?it/s]

  ↳ streamed 20 for 2018 (file size now 20127.4 KB)
  ↳ streamed 40 for 2018 (file size now 20409.2 KB)
  ↳ streamed 60 for 2018 (file size now 20664.1 KB)
  ↳ streamed 80 for 2018 (file size now 20947.5 KB)
  ↳ streamed 100 for 2018 (file size now 21239.8 KB)
  ↳ streamed 120 for 2018 (file size now 21531.0 KB)
  ↳ streamed 140 for 2018 (file size now 21853.4 KB)
  ↳ streamed 160 for 2018 (file size now 22076.3 KB)
  ↳ streamed 180 for 2018 (file size now 22359.3 KB)
  ↳ streamed 200 for 2018 (file size now 22664.1 KB)
  ↳ streamed 220 for 2018 (file size now 23020.8 KB)
  ↳ streamed 240 for 2018 (file size now 23277.0 KB)
  ↳ streamed 260 for 2018 (file size now 23596.0 KB)
  ↳ streamed 280 for 2018 (file size now 23899.6 KB)
  ↳ streamed 300 for 2018 (file size now 24176.7 KB)
  ↳ streamed 320 for 2018 (file size now 24472.1 KB)
✓ 2018: added 321 (running total rows: 1609)  →  mlb_postseason_transcripts_2014_2019.csv
2019: 18 series pages, 0 day pages
2019: crawling 96 day/interview

2019 pages:   0%|          | 0/96 [00:00<?, ?it/s]

  ↳ streamed 20 for 2019 (file size now 24782.4 KB)
  ↳ streamed 40 for 2019 (file size now 25133.3 KB)
  ↳ streamed 60 for 2019 (file size now 25434.5 KB)
  ↳ streamed 80 for 2019 (file size now 25733.2 KB)
  ↳ streamed 100 for 2019 (file size now 26047.5 KB)
  ↳ streamed 120 for 2019 (file size now 26363.8 KB)
  ↳ streamed 140 for 2019 (file size now 26616.9 KB)
  ↳ streamed 160 for 2019 (file size now 26932.6 KB)
  ↳ streamed 180 for 2019 (file size now 27236.4 KB)
  ↳ streamed 200 for 2019 (file size now 27520.6 KB)
  ↳ streamed 220 for 2019 (file size now 27810.4 KB)
  ↳ streamed 240 for 2019 (file size now 28119.6 KB)
  ↳ streamed 260 for 2019 (file size now 28439.0 KB)
  ↳ streamed 280 for 2019 (file size now 28757.7 KB)
  ↳ streamed 300 for 2019 (file size now 29009.7 KB)
  ↳ streamed 320 for 2019 (file size now 29276.8 KB)
  ↳ streamed 340 for 2019 (file size now 29590.3 KB)
  ↳ streamed 360 for 2019 (file size now 29820.0 KB)
✓ 2019: added 379 (running total rows: 1988)  →  m

In [ ]:

# Team Matching

import pandas as pd, re, json, time, requests
from tqdm.notebook import tqdm
from math import ceil

enriched = pd.read_csv(ENRICHED_CSV, dtype=str)
for c in ["year","series","matchup","date_iso","interviewee","team_a","team_b","transcript","interview_url"]:
    if c not in enriched.columns: enriched[c] = ""

# manual overrides
manual = {}
if os.path.exists(MANUAL_OVERRIDES):
    mo = pd.read_csv(MANUAL_OVERRIDES, dtype=str)
    for _, r in mo.iterrows():
        manual[(str(r["interviewee"]).strip().title(), str(r["year"]).strip())] = str(r["team"]).strip().upper()

# cache
try:
    with open(NAME_CACHE_JSON,"r",encoding="utf-8") as f:
        name_cache = json.load(f)
except:
    name_cache = {}

# team alias map again (canonical)
TEAM_ALIASES = {
    "ARIZONA DIAMONDBACKS": ["DIAMONDBACKS","ARIZONA"],
    "ATLANTA BRAVES": ["ATLANTA","BRAVES"],
    "BALTIMORE ORIOLES": ["BALTIMORE","ORIOLES","O'S","OS"],
    "BOSTON RED SOX": ["BOSTON","RED SOX","REDSOX"],
    "CHICAGO CUBS": ["CHICAGO CUBS","CUBS"],
    "CHICAGO WHITE SOX": ["CHICAGO WHITE SOX","WHITE SOX","WHITESOX","SOX"],
    "CINCINNATI REDS": ["CINCINNATI","REDS"],
    "CLEVELAND GUARDIANS": ["CLEVELAND","GUARDIANS","INDIANS"],
    "COLORADO ROCKIES": ["COLORADO","ROCKIES"],
    "DETROIT TIGERS": ["DETROIT","TIGERS"],
    "HOUSTON ASTROS": ["HOUSTON","ASTROS"],
    "KANSAS CITY ROYALS": ["KANSAS CITY","ROYALS","KC"],
    "LOS ANGELES ANGELS": ["LOS ANGELES ANGELS","ANGELS","LAA","ANAHEIM"],
    "LOS ANGELES DODGERS": ["LOS ANGELES DODGERS","DODGERS","LAD"],
    "MIAMI MARLINS": ["MIAMI","MARLINS","FLORIDA MARLINS"],
    "MILWAUKEE BREWERS": ["MILWAUKEE","BREWERS"],
    "MINNESOTA TWINS": ["MINNESOTA","TWINS"],
    "NEW YORK METS": ["NEW YORK METS","METS","NYM"],
    "NEW YORK YANKEES": ["NEW YORK YANKEES","YANKEES","NYY"],
    "OAKLAND ATHLETICS": ["OAKLAND","ATHLETICS","A'S","AS"],
    "PHILADELPHIA PHILLIES": ["PHILADELPHIA","PHILLIES","PHILS"],
    "PITTSBURGH PIRATES": ["PITTSBURGH","PIRATES","BUCS","BUCCOS"],
    "SAN DIEGO PADRES": ["SAN DIEGO","PADRES"],
    "SAN FRANCISCO GIANTS": ["SAN FRANCISCO","GIANTS"],
    "SEATTLE MARINERS": ["SEATTLE","MARINERS","MS"],
    "ST. LOUIS CARDINALS": ["ST. LOUIS","ST LOUIS","CARDINALS","CARDS"],
    "TAMPA BAY RAYS": ["TAMPA BAY","RAYS","DEVIL RAYS"],
    "TEXAS RANGERS": ["TEXAS","RANGERS"],
    "TORONTO BLUE JAYS": ["TORONTO","BLUE JAYS","JAYS"],
    "WASHINGTON NATIONALS": ["WASHINGTON","NATIONALS","NATS"],
}
ALIAS_TO_CANON = {alias.upper():team for team,aliases in TEAM_ALIASES.items() for alias in [team]+aliases}

def canon_team(t):
    return ALIAS_TO_CANON.get(str(t).upper(), str(t).title()) if pd.notna(t) else None

# build matchup teams (from parsed columns)
def split_matchup(m):
    if isinstance(m,str) and " vs " in m:
        a,b = m.split(" vs ",1)
        return a.strip(), b.strip()
    return None, None

teamsAB = enriched["matchup"].apply(split_matchup)
enriched["match_team_a"] = teamsAB.apply(lambda x: x[0])
enriched["match_team_b"] = teamsAB.apply(lambda x: x[1])

# --- Wikipedia helper functions (reads Teams row) ---
sess = requests.Session()
sess.headers.update({"User-Agent":"MLB-TeamResolver/2.1"})

def wiki_search(name):
    try:
        r = sess.get("https://en.wikipedia.org/w/api.php",
                     params={"action":"query","list":"search","srsearch":name,"format":"json","srlimit":1},
                     timeout=15)
        return r.json().get("query",{}).get("search",[{}])[0].get("title")
    except Exception:
        return None

def wiki_infobox_html(title):
    try:
        r = sess.get("https://en.wikipedia.org/w/api.php",
                     params={"action":"parse","page":title,"prop":"text","formatversion":2,"format":"json"},
                     timeout=20)
        return r.json().get("parse",{}).get("text","")
    except Exception:
        return None

def extract_teams(html):
    if not html: return []
    m = re.search(r"Teams</th>\s*<td[^>]*>(.*?)</td>", html, flags=re.I|re.S)
    if not m: return []
    text = re.sub(r"<[^>]+>"," ", m.group(1))
    text = re.sub(r"\s+"," ", text).strip()
    # split by bullet/semi
    chunks = re.split(r"[;•|]", text)
    return [c.strip() for c in chunks if c.strip()]

def pick_from_teams(teams_texts, matchup_set):
    # choose the first that mentions one of the matchup teams (by last word or full name)
    for t in teams_texts:
        for mt in matchup_set:
            if not mt: continue
            if mt.split()[-1].lower() in t.lower() or mt.lower() in t.lower():
                return mt
    return None

def resolve_team(name, year, teamA, teamB):
    nm = str(name).strip()
    yr = re.search(r"(\d{4})", str(year))
    yr = yr.group(1) if yr else None
    if not nm or not yr: return None

    # manual override first
    key_override = (nm.title(), yr)
    if key_override in manual:
        return manual[key_override]

    cache_key = f"{nm.title()}|||{yr}"
    if cache_key in name_cache:
        return name_cache[cache_key]

    title = wiki_search(nm)
    if not title:
        name_cache[cache_key] = None; return None
    html = wiki_infobox_html(title)
    teams_texts = extract_teams(html)
    choice = pick_from_teams(teams_texts, {teamA, teamB})
    name_cache[cache_key] = choice
    time.sleep(0.25)
    return choice

# assign
assigned = []
for r in tqdm(enriched.itertuples(index=False), total=len(enriched)):
    team = resolve_team(getattr(r,"interviewee",""), getattr(r,"year",""),
                        getattr(r,"match_team_a",None), getattr(r,"match_team_b",None))
    assigned.append(team)

enriched["assigned_team"] = assigned

# persist cache
with open(NAME_CACHE_JSON,"w",encoding="utf-8") as f:
    json.dump(name_cache,f,ensure_ascii=False,indent=2)

# --- Group and export (only rows with assigned_team) ---
to_group = enriched[enriched["assigned_team"].notna()].copy()

# Guarantee single 1-D columns & clean strings
for col in ["year","series","matchup","date_iso","assigned_team","interview_url","transcript"]:
    if col not in to_group.columns: to_group[col] = ""
to_group["year"] = to_group["year"].astype(str).str.extract(r"(\d{4})")[0]

group_cols = ["year","series","matchup","date_iso","assigned_team"]
grouped = (
    to_group
    .groupby(group_cols, dropna=False, as_index=False)
    .agg(
        interviews_count=("interview_url","count"),
        combined_transcript=("transcript", lambda s: "\n\n".join(s))
    )
)
grouped["combined_word_count"] = grouped["combined_transcript"].str.split().str.len()

# Excel overflow-safe split
MAX_CELL = 32767
parts_needed = grouped["combined_transcript"].apply(lambda x: ceil(len((x or ""))/MAX_CELL))
max_parts = int(parts_needed.max()) if len(parts_needed) else 1
for j in range(1, max_parts+1):
    grouped[f"transcript_part{j}"] = ""
for i, s in grouped["combined_transcript"].items():
    if s:
        for j in range(0, ceil(len(s)/MAX_CELL)):
            grouped.at[i, f"transcript_part{j+1}"] = s[j*MAX_CELL:(j+1)*MAX_CELL]

grouped.to_excel(GROUPED_XLSX, index=False)
grouped.to_parquet(GROUPED_PARQUET, index=False)
enriched.to_csv(ENRICHED_CSV, index=False)  # overwrite with assigned_team column

print("✅ Wrote:")
print(" -", ENRICHED_CSV)
print(" -", GROUPED_XLSX)
print(" -", GROUPED_PARQUET)

# Also save unassigned for quick review
enriched[enriched["assigned_team"].isna()][["year","series","matchup","date_iso","interviewee","interview_url"]].to_csv(
    "team_unassigned_rows.csv", index=False
)
print("Also wrote team_unassigned_rows.csv (for manual review).")


  0%|          | 0/3646 [00:00<?, ?it/s]

✅ Wrote:
 - transcripts_enriched.csv
 - team_game_combined.xlsx
 - team_game_combined.parquet
Also wrote team_unassigned_rows.csv (for manual review).


In [ ]:
# Rosters, Block 1: Install & Import
!pip install -q thefuzz
import pandas as pd
from thefuzz import process
import os

print("Libraries installed.")

Libraries installed.


In [ ]:
#Rosters, Block 2: Load All Data & Build Smart Lookups (FINAL FIX)

# --- Define file paths (they are now in the main directory) ---
PEOPLE_CSV = 'People.csv'
BATTING_CSV = 'Batting.csv'
PITCHING_CSV = 'Pitching.csv'
MANAGERS_CSV = 'Managers.csv'

# --- Confirm your file names ---
YOUR_FILE_NAME = 'transcripts_enriched.csv'  # <-- I've kept this from your last message
PLAYER_NAME_COLUMN = 'interviewee'
YEAR_COLUMN = 'year'
# -----------------------------

try:
    # --- Load Your Data ---
    user_df = pd.read_csv(YOUR_FILE_NAME)
    print(f"Successfully loaded '{YOUR_FILE_NAME}'. Original shape: {user_df.shape}")

    # 1. Clean the interviewee name
    user_df[PLAYER_NAME_COLUMN] = user_df[PLAYER_NAME_COLUMN].str.strip()

    # 2. Deduplicate
    dedupe_cols = ['interviewee', 'date_iso', 'word_count']
    rows_before = len(user_df)
    user_df = user_df.drop_duplicates(subset=dedupe_cols, keep='first')
    rows_after = len(user_df)
    print(f"Deduplicated: Removed {rows_before - rows_after} rows. New shape: {user_df.shape}")

    # --- Build Lahman Lookups from Uploaded CSVs ---
    print("\nBuilding smart name lookup...")
    people_df = pd.read_csv(PEOPLE_CSV)

    all_names_df = people_df[['playerID', 'nameFirst', 'nameLast', 'debut']].copy()
    all_names_df['Name'] = all_names_df['nameFirst'] + ' ' + all_names_df['nameLast']
    all_names_df = all_names_df.sort_values('debut', ascending=False)

    people_lookup = all_names_df.drop_duplicates('Name').set_index('Name')['playerID']
    all_lahman_names = people_lookup.index.tolist()
    print(f"Created 'people' lookup with {len(all_lahman_names)} unique names.")

    # --- Build UP-TO-DATE 'team' lookup (THE FIX IS HERE) ---
    print("Building up-to-date 'team' lookup (prioritizing *last* team)...")
    batting_df = pd.read_csv(BATTING_CSV)
    pitching_df = pd.read_csv(PITCHING_CSV)
    managers_df = pd.read_csv(MANAGERS_CSV)

    # Get player stints (using 'stint' column)
    player_stints = pd.concat([
        batting_df[['playerID', 'yearID', 'teamID', 'stint']],
        pitching_df[['playerID', 'yearID', 'teamID', 'stint']]
    ])

    # Get manager stints (using 'inseason' as the 'stint')
    manager_stints = managers_df[['playerID', 'yearID', 'teamID', 'inseason']].rename(
        columns={'inseason': 'stint'}
    )

    # Combine all stints
    all_stints = pd.concat([player_stints, manager_stints])

    # --- THIS IS THE KEY CHANGE ---
    # We sort by 'stint' (descending) to get the highest stint number (the last team)
    # Then we drop duplicates, keeping the first one (which is now the last team)
    final_team = all_stints.sort_values('stint', ascending=False).drop_duplicates(['playerID', 'yearID'])

    team_lookup = final_team.set_index(['playerID', 'yearID'])['teamID']
    print("Master lookups are built and ready.")

except FileNotFoundError as e:
    print(f"--- !!! FILE NOT FOUND ERROR !!! ---")
    print(f"Could not find file: {e.filename}")
    print("Please make sure you have uploaded all 5 files to Colab.")
except Exception as e:
    print(f"An error occurred: {e}")

Successfully loaded 'transcripts_enriched.csv'. Original shape: (3646, 14)
Deduplicated: Removed 10 rows. New shape: (3636, 14)

Building smart name lookup...
Created 'people' lookup with 20566 unique names.
Building up-to-date 'team' lookup (prioritizing *last* team)...
Master lookups are built and ready.


In [ ]:
#Rosters Block 3: Define the Fuzzy Matching Function

def find_team_fuzzy(row):
    # Get the name and year from the row
    name_to_find = row[PLAYER_NAME_COLUMN]
    year_to_find = row[YEAR_COLUMN]

    # 1. Check for a perfect match first (fast)
    player_id = people_lookup.get(name_to_find)

    # 2. If no perfect match, use fuzzy matching
    if player_id is None:
        try:
            best_match = process.extractOne(name_to_find, all_lahman_names, score_cutoff=85)

            if best_match:
                matched_name = best_match[0]
                player_id = people_lookup.get(matched_name)
        except:
            return None

    # 3. If we have a player_id, find the team
    if player_id:
        team = team_lookup.get((player_id, year_to_find))
        return team

    # 4. If all else fails, return None
    return None

print("Matcher function defined.")

Matcher function defined.


In [ ]:
#Rosters Block 4: Apply Function, Review, and Save

print("Applying fuzzy match to all rows... This may take a minute.")

# .apply() runs the function on every single row
user_df['Team'] = user_df.apply(find_team_fuzzy, axis=1)

print("Matching complete.")

# --- Review ---
print("\n--- Review Final Data ---")
print(user_df.head(20))

# Show a summary
total_rows = len(user_df)
matched_rows = user_df['Team'].notna().sum()
unmatched_rows = user_df['Team'].isna().sum()

print(f"\n--- FINAL Match Summary (Manual Upload) ---")
print(f"Total Rows: {total_rows}")
print(f"Rows with Team Found: {matched_rows}")
print(f"Rows with No Team Found: {unmatched_rows}")

if unmatched_rows > 0:
    print("\nNOTE: Remaining unmatched rows are likely non-players (coaches, execs),")
    print("names with very bad misspellings, or players inactive that year.")
    print("\nSee 'unmatched_rows.csv' for a list of these.")

    unmatched_df = user_df[user_df['Team'].isna()]
    unmatched_df.to_csv('unmatched_rows.csv', index=False)


# Save the final, merged data to a new CSV
OUTPUT_FILE = 'your_data_with_teams_FINAL.csv'
user_df.to_csv(OUTPUT_FILE, index=False)

print(f"\nSuccessfully saved your new file as: {OUTPUT_FILE}")
print("You can find it in the Colab sidebar (folder icon) and download it.")

Applying fuzzy match to all rows... This may take a minute.
Matching complete.

--- Review Final Data ---
                                        interview_url  \
0   https://www.asapsports.com/show_interview.php?...   
1   https://www.asapsports.com/show_interview.php?...   
2   https://www.asapsports.com/show_interview.php?...   
3   https://www.asapsports.com/show_interview.php?...   
4   https://www.asapsports.com/show_interview.php?...   
5   https://www.asapsports.com/show_interview.php?...   
6   https://www.asapsports.com/show_interview.php?...   
7   https://www.asapsports.com/show_interview.php?...   
8   https://www.asapsports.com/show_interview.php?...   
9   https://www.asapsports.com/show_interview.php?...   
10  https://www.asapsports.com/show_interview.php?...   
11  https://www.asapsports.com/show_interview.php?...   
12  https://www.asapsports.com/show_interview.php?...   
13  https://www.asapsports.com/show_interview.php?...   
14  https://www.asapsports.com/show_int

In [ ]:
#Rosters Block 5: Drop Unnecessary Columns (Revised)

# This will DROP 'team_a', 'team_b', and 'assigned_team'
# It will keep the 'match_team' columns.
columns_to_drop = ['team_a', 'team_b', 'assigned_team']

# To keep 'team_a' and 'team_b' (and drop the 'match_team' columns instead),
# uncomment the line below:
# columns_to_drop = ['match_team_a', 'match_team_b', 'assigned_team']

# --- RUN THE CODE ---
try:
    # Check if the dataframe exists
    if 'user_df' not in locals():
        print("Error: 'user_df' not found.")
        print("Please re-run Block 2 to load your data, then run this block again.")

    # Check if the columns exist before trying to drop
    elif all(col in user_df.columns for col in columns_to_drop):
        user_df = user_df.drop(columns=columns_to_drop)
        print(f"Successfully dropped the columns: {columns_to_drop}")
        print("Here is a preview of the new data structure:")
        print(user_df.head())

        # Save the file again with the new, cleaner data
        OUTPUT_FILE = 'your_data_FINAL_cleaned.csv'
        user_df.to_csv(OUTPUT_FILE, index=False)
        print(f"\nSuccessfully saved the cleaned data to: {OUTPUT_FILE}")
        print("You can now download this new file.")
    else:
        # Check which columns are missing
        missing_cols = [col for col in columns_to_drop if col not in user_df.columns]
        print(f"Error: The following columns were not found: {missing_cols}")
        print("They might have been dropped already, or the names are incorrect.")
        print("Current columns:", user_df.columns.tolist())

except Exception as e:
    print(f"An error occurred: {e}")

Error: The following columns were not found: ['team_a', 'team_b']
They might have been dropped already, or the names are incorrect.
Current columns: ['interview_url', 'title', 'word_count', 'transcript', 'year', 'series', 'matchup', 'date_iso', 'interviewee', 'match_team_a', 'match_team_b', 'assigned_team', 'Team']


In [ ]:
#Rosters Block 6: Drop the last remaining column

# 'team_a' and 'team_b' are already gone.
# We only need to drop 'assigned_team'.
columns_to_drop = ['assigned_team']

# --- RUN THE CODE ---
try:
    # Check if the dataframe exists
    if 'user_df' not in locals():
        print("Error: 'user_df' not found.")
        print("Please re-run Block 2 to load your data, then run this block again.")

    # Check if the columns exist before trying to drop
    elif all(col in user_df.columns for col in columns_to_drop):
        user_df = user_df.drop(columns=columns_to_drop)
        print(f"Successfully dropped the columns: {columns_to_drop}")
        print("Here is a preview of the new data structure:")
        print(user_df.head())

        # Save the file again with the new, cleaner data
        OUTPUT_FILE = 'your_data_FINAL_cleaned.csv'
        user_df.to_csv(OUTPUT_FILE, index=False)
        print(f"\nSuccessfully saved the cleaned data to: {OUTPUT_FILE}")
        print("You can now download this new file.")
    else:
        # Check which columns are missing
        missing_cols = [col for col in columns_to_drop if col not in user_df.columns]
        print(f"Error: The following columns were not found: {missing_cols}")
        print("They might have been dropped already, or the names are incorrect.")
        print("Current columns:", user_df.columns.tolist())

except Exception as e:
    print(f"An error occurred: {e}")

Successfully dropped the columns: ['assigned_team']
Here is a preview of the new data structure:
                                       interview_url  \
0  https://www.asapsports.com/show_interview.php?...   
1  https://www.asapsports.com/show_interview.php?...   
2  https://www.asapsports.com/show_interview.php?...   
3  https://www.asapsports.com/show_interview.php?...   
4  https://www.asapsports.com/show_interview.php?...   

                                               title  word_count  \
0  ASAP Sports Transcripts - Baseball - 2014 - NL...        3711   
1  ASAP Sports Transcripts - Baseball - 2014 - NL...        3749   
2  ASAP Sports Transcripts - Baseball - 2014 - NL...        3159   
3  ASAP Sports Transcripts - Baseball - 2014 - NL...        1447   
4  ASAP Sports Transcripts - Baseball - 2014 - NL...        2983   

                                          transcript  year  \
0  Q.Â Would Lackey or Lynn be available tonight ...  2014   
1  Q.Â To have originally come up

In [ ]:
#Playoff Roster Creation
# %%capture
!pip -q install pandas requests tqdm

# %%
import time, json, requests, pandas as pd
from tqdm.auto import tqdm

# -----------------------------------
# Config
# -----------------------------------
YEARS = [y for y in range(2014, 2025) if y != 2020]  # 2014–2024, skip 2020
STATS_BASE = "https://statsapi.mlb.com/api/v1"
HEADERS = {"User-Agent":"Mozilla/5.0 (compatible; postseason-rosters/2.0)"}

# Map MLB StatsAPI teamId -> Retrosheet-style 3-letter codes (to join your CSV)
TEAMID_TO_RETRO = {
    110:"BAL",111:"BOS",147:"NYA",139:"TBA",141:"TOR",
    145:"CHA",114:"CLE",116:"DET",118:"KCA",142:"MIN",
    117:"HOU",108:"LAA",133:"OAK",136:"SEA",140:"TEX",
    144:"ATL",146:"MIA",121:"NYN",143:"PHI",120:"WAS",
    112:"CHN",113:"CIN",158:"MIL",134:"PIT",138:"SLN",
    109:"ARI",115:"COL",119:"LAN",135:"SDN",137:"SFN",
}

def series_tag_from_desc(desc: str) -> str:
    if not isinstance(desc, str): return ""
    d = desc.lower()
    if "wild" in d: return "wc"
    if "division" in d: return "ds"
    if "championship" in d: return "lcs"
    if "world series" in d or "ws" in d: return "ws"
    return ""

# -----------------------------------
# Tiny HTTP client with caching/retries
# -----------------------------------
class Http:
    def __init__(self):
        self.sess = requests.Session()
        self.cache = {}
    def get_json(self, url, params=None, timeout=25, retries=3, backoff=0.6):
        key = (url, tuple(sorted((params or {}).items())))
        if key in self.cache:
            return self.cache[key]
        last = None
        for i in range(retries):
            try:
                r = self.sess.get(url, params=params, headers=HEADERS, timeout=timeout)
                r.raise_for_status()
                js = r.json()
                self.cache[key] = js
                return js
            except Exception as e:
                last = e
                time.sleep(backoff * (i+1))
        raise last

http = Http()

# -----------------------------------
# 1) Pull all postseason games for each year
# Prefer /schedule/postseason, fall back to /schedule with gameTypes F,D,L,W
# -----------------------------------
def get_postseason_games_year(year: int):
    games = []
    # Preferred endpoint
    try:
        js = http.get_json(f"{STATS_BASE}/schedule/postseason", params={"season": year, "sportId": 1})
        for d in js.get("dates", []):
            for g in d.get("games", []):
                if not g.get("gamePk"): continue
                games.append({
                    "year": year,
                    "date": g.get("officialDate") or (g.get("gameDate","")[:10]),
                    "gamePk": int(g["gamePk"]),
                    "homeId": g.get("teams",{}).get("home",{}).get("team",{}).get("id"),
                    "awayId": g.get("teams",{}).get("away",{}).get("team",{}).get("id"),
                    "series_desc": g.get("seriesDescription") or g.get("seriesDescriptionShort") or "",
                })
    except Exception:
        pass

    if games:
        return games

    # Fallback endpoint with explicit gameTypes
    try:
        js = http.get_json(f"{STATS_BASE}/schedule", params={"sportId":1, "season":year, "gameTypes":"F,D,L,W"})
        for d in js.get("dates", []):
            for g in d.get("games", []):
                if not g.get("gamePk"): continue
                games.append({
                    "year": year,
                    "date": g.get("officialDate") or (g.get("gameDate","")[:10]),
                    "gamePk": int(g["gamePk"]),
                    "homeId": g.get("teams",{}).get("home",{}).get("team",{}).get("id"),
                    "awayId": g.get("teams",{}).get("away",{}).get("team",{}).get("id"),
                    "series_desc": g.get("seriesDescription") or g.get("seriesDescriptionShort") or "",
                })
    except Exception:
        pass

    return games

all_games = []
for y in YEARS:
    all_games.extend(get_postseason_games_year(y))

games_df = pd.DataFrame(all_games).drop_duplicates()
if games_df.empty:
    raise RuntimeError("No postseason games found from the API endpoints.")

# Keep only games where both team IDs map cleanly
mask = games_df["homeId"].isin(TEAMID_TO_RETRO) & games_df["awayId"].isin(TEAMID_TO_RETRO)
games_df = games_df[mask].reset_index(drop=True)

# -----------------------------------
# 2) Boxscores -> full game roster (includes DNPs)
# Correct JSON path: js['teams']['home'|'away']['players']
# -----------------------------------
def fetch_boxscore_roster(game_pk: int):
    js = http.get_json(f"{STATS_BASE}/game/{game_pk}/boxscore")
    teams = js.get("teams", {}) or {}
    out = []
    for side in ("home","away"):
        tnode = teams.get(side, {}) or {}
        tid = (tnode.get("team") or {}).get("id")
        players = (tnode.get("players") or {})
        for pdata in players.values():
            person = pdata.get("person", {}) or {}
            pos = pdata.get("position", {}) or {}
            out.append({
                "teamId": tid,
                "mlb_player_id": person.get("id"),
                "fullName": person.get("fullName"),
                "primaryNumber": pdata.get("jerseyNumber"),
                "position_code": pos.get("code"),
                "position_name": pos.get("name"),
            })
    return out

# -----------------------------------
# 3) Build YEAR × TEAM union rosters (no daily/series outputs)
# -----------------------------------
# Use dict: {(year, teamId) -> {player_id -> info}}
year_team_players = {}

print("Fetching boxscores & unioning to year-level rosters…")
for _, g in tqdm(games_df.iterrows(), total=len(games_df)):
    pk = int(g["gamePk"])
    y = int(g["year"])
    try:
        roster = fetch_boxscore_roster(pk)
    except Exception as e:
        # log and continue
        continue

    for tid in (g["homeId"], g["awayId"]):
        bucket = year_team_players.setdefault((y, int(tid)), {})
        for r in roster:
            if r.get("teamId") != tid:
                continue
            pid = r.get("mlb_player_id")
            if not pid:
                continue
            # keep first seen info (names/positions)
            if pid not in bucket:
                bucket[pid] = {
                    "mlb_player_id": int(pid),
                    "fullName": r.get("fullName"),
                    "primaryNumber": r.get("primaryNumber"),
                    "position_code": r.get("position_code"),
                    "position_name": r.get("position_name"),
                }
    time.sleep(0.02)

# Materialize rows
rows = []
for (year, teamId), plist in year_team_players.items():
    team_code = TEAMID_TO_RETRO.get(int(teamId))
    for info in plist.values():
        rows.append({
            "year": year,
            "teamId": int(teamId),
            "team": team_code,
            **info
        })

out = pd.DataFrame(rows).drop_duplicates().sort_values(["year","team","fullName"]).reset_index(drop=True)

# Save: one roster per team per year (union of all postseason games that year)
out_path = "/content/mlb_postseason_year_team_rosters_2014_2024.csv"
out.to_csv(out_path, index=False)
print(f"✅ Saved postseason year-level rosters to {out_path} (rows: {len(out):,})")

# peek a few lines
print(out.head(20).to_string(index=False))


Fetching boxscores & unioning to year-level rosters…


  0%|          | 0/374 [00:00<?, ?it/s]

✅ Saved postseason year-level rosters to /content/mlb_postseason_year_team_rosters_2014_2024.csv (rows: 2,871)
 year  teamId team  mlb_player_id         fullName primaryNumber position_code     position_name
 2014     110  BAL         430945       Adam Jones            10             8        Outfielder
 2014     110  BAL         457477 Alejandro De Aza            12             7        Outfielder
 2014     110  BAL         453192    Andrew Miller            48             1           Pitcher
 2014     110  BAL         542960       Brad Brach            35             1           Pitcher
 2014     110  BAL         451085     Brian Matusz            17             1           Pitcher
 2014     110  BAL         502032       Bud Norris            25             1           Pitcher
 2014     110  BAL         543376     Caleb Joseph            36             2           Catcher
 2014     110  BAL         501957    Chris Tillman            30             1           Pitcher
 2014     110  B

In [ ]:
# Win Loss identification
INPUT_CSV  = "/content/your_data_FINAL_cleaned.csv"
OUTPUT_CSV = "/content/your_data_with_team_won.csv"

# === CODE ===
import pandas as pd, numpy as np, re, time, requests
from tqdm.auto import tqdm

# -------- 1) Load & clean your data --------
df = pd.read_csv(INPUT_CSV, dtype=str, keep_default_na=False)

# Drop rows with no team (execs/umpires)
df["Team"] = df["Team"].str.strip()
df["Team"].replace({"": np.nan}, inplace=True)
df = df.dropna(subset=["Team"]).copy()

# Safe year extraction (handles accidental text in the column)
def safe_year(x):
    m = re.search(r"(20[0-3]\d)", str(x))
    return int(m.group(1)) if m else np.nan

df["year"] = df["year"].apply(safe_year)
df = df.dropna(subset=["year"]).copy()
df["year"] = df["year"].astype(int)

# Parse dates; your screenshot showed MM/DD/YY
date_col = "date_iso" if "date_iso" in df.columns else "date"
def parse_mdy2(x):
    try:
        return pd.to_datetime(x, format="%m/%d/%y", errors="coerce")
    except Exception:
        return pd.to_datetime(x, errors="coerce")
df["date"] = df[date_col].apply(parse_mdy2)

# Normalize team codes (to Retrosheet-style)
TEAM_ALIASES = {
    "ANA":"LAA","FLA":"MIA","TBD":"TBA","TB":"TBA","WSH":"WAS",
    "LAD":"LAN","SDP":"SDN","SFG":"SFN","CHC":"CHN","STL":"SLN",
    "NYY":"NYA","NYM":"NYN","KC":"KCA","SF":"SFN","LA":"LAN"
}
def norm_team(code):
    if pd.isna(code): return np.nan
    code = str(code).strip().upper()
    return TEAM_ALIASES.get(code, code)
df["Team"] = df["Team"].map(norm_team)

VALID_YEARS = {2014,2015,2016,2017,2018,2019,2021,2022,2023,2024}
df = df[df["year"].isin(VALID_YEARS)].copy()

# -------- 2) Build postseason Win/Loss lookup from MLB StatsAPI --------
STATS_BASE = "https://statsapi.mlb.com/api/v1"
HEADERS = {"User-Agent":"Mozilla/5.0 (team-won/solid-1.0)"}

# MLB teamId -> Retrosheet-style 3-letter code
TEAMID_TO_RETRO = {
    110:"BAL",111:"BOS",147:"NYA",139:"TBA",141:"TOR",
    145:"CHA",114:"CLE",116:"DET",118:"KCA",142:"MIN",
    117:"HOU",108:"LAA",133:"OAK",136:"SEA",140:"TEX",
    144:"ATL",146:"MIA",121:"NYN",143:"PHI",120:"WAS",
    112:"CHN",113:"CIN",158:"MIL",134:"PIT",138:"SLN",
    109:"ARI",115:"COL",119:"LAN",135:"SDN",137:"SFN"
}

def get_json(url, params=None, retries=3, backoff=0.6):
    last = None
    for i in range(retries):
        try:
            r = requests.get(url, params=params, headers=HEADERS, timeout=30)
            r.raise_for_status()
            return r.json()
        except Exception as e:
            last = e
            time.sleep(backoff*(i+1))
    raise last

def game_results_for_year(year: int):
    """Return rows: year, date, team_code, result ('Win'/'Loss') for postseason games."""
    rows = []

    # Preferred: official postseason schedule
    data = None
    try:
        data = get_json(f"{STATS_BASE}/schedule/postseason", params={"season":year,"sportId":1})
    except Exception:
        data = None

    # Fallback: schedule with explicit postseason gameTypes
    if not data or not data.get("dates"):
        try:
            data = get_json(f"{STATS_BASE}/schedule", params={"season":year,"sportId":1,"gameTypes":"F,D,L,W"})
        except Exception:
            data = {"dates":[]}

    for d in data.get("dates", []):
        for g in d.get("games", []):
            if not g.get("gamePk"):
                continue
            # extract IDs & date
            date = pd.to_datetime(g.get("officialDate") or g.get("gameDate","")[:10], errors="coerce")
            home = g.get("teams", {}).get("home", {}) or {}
            away = g.get("teams", {}).get("away", {}) or {}
            hteam = (home.get("team") or {}).get("id")
            ateam = (away.get("team") or {}).get("id")
            if not (isinstance(hteam,int) and isinstance(ateam,int)):
                continue
            h_code = TEAMID_TO_RETRO.get(hteam)
            a_code = TEAMID_TO_RETRO.get(ateam)
            if not (h_code and a_code):
                continue

            # Preferred: use isWinner flags when present
            h_win = home.get("isWinner")
            a_win = away.get("isWinner")

            if isinstance(h_win, bool) and isinstance(a_win, bool):
                rows.append({"year":year, "date":date, "team":h_code, "result":"Win" if h_win else "Loss"})
                rows.append({"year":year, "date":date, "team":a_code, "result":"Win" if a_win else "Loss"})
                continue

            # Fallback: pull linescore for final runs
            try:
                ls = get_json(f"{STATS_BASE}/game/{int(g['gamePk'])}/linescore")
                teams = ls.get("teams", {})
                hr = teams.get("home", {}).get("runs")
                ar = teams.get("away", {}).get("runs")
                if hr is None or ar is None:
                    continue
                if hr > ar:
                    rows.append({"year":year, "date":date, "team":h_code, "result":"Win"})
                    rows.append({"year":year, "date":date, "team":a_code, "result":"Loss"})
                elif ar > hr:
                    rows.append({"year":year, "date":date, "team":a_code, "result":"Win"})
                    rows.append({"year":year, "date":date, "team":h_code, "result":"Loss"})
            except Exception:
                # if even linescore fails, skip the game
                continue

    return rows

years_needed = sorted(df["year"].unique())
all_rows = []
for y in tqdm(years_needed, desc="Fetching postseason results"):
    if y == 2020:   # your dataset excludes 2020
        continue
    all_rows.extend(game_results_for_year(y))
    time.sleep(0.2)

results = pd.DataFrame(all_rows).dropna()
results["date"] = pd.to_datetime(results["date"], errors="coerce")

# -------- 3) Merge & label --------
merged = df.merge(results, left_on=["year","Team","date"], right_on=["year","team","date"], how="left")
merged["team_won"] = merged["result"].fillna("No Game")
merged.drop(columns=[c for c in ["team","result"] if c in merged.columns], inplace=True)

# -------- 4) Save --------
merged.to_csv(OUTPUT_CSV, index=False)
print(f"✅ Saved with Win/Loss labels: {OUTPUT_CSV}")
print(merged["team_won"].value_counts(dropna=False))


/tmp/ipython-input-2787576426.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Team"].replace({"": np.nan}, inplace=True)


Fetching postseason results:   0%|          | 0/10 [00:00<?, ?it/s]

✅ Saved with Win/Loss labels: /content/your_data_with_team_won.csv
team_won
Win        1639
Loss       1144
No Game     815
Name: count, dtype: int64


In [ ]:
#Deduplication, cleaning of transcripts + Word Count and Mentions
import pandas as pd, numpy as np, re, unicodedata, os

# ==== INPUTS (edit paths if needed) ====
INTERVIEWS_IN = "/content/your_data_with_team_won_games_only.xlsx"
TEAM_DATE_IN  = "/content/team_date_teammate_mentions_with_series.csv"  # use CSV if present, else switch to .xlsx

# ==== OUTPUTS ====
TEAM_DATE_OUT_CSV  = "/content/team_date_teammate_mentions_with_series_cleanwords.csv"
TEAM_DATE_OUT_XLSX = "/content/team_date_teammate_mentions_with_series_cleanwords.xlsx"

# ---------- helpers ----------
def normalize_text(s: str) -> str:
    if not isinstance(s, str): return ""
    s = unicodedata.normalize("NFKC", s)
    s = s.replace("\xa0", " ").replace("Â", "")
    s = re.sub(r"[ \t]+", " ", s)
    s = s.replace("\r", "\n")
    return s.strip()

def strip_questions_and_keep_interviewee(text: str, interviewee_name: str) -> str:
    """Remove Q./MODERATOR blocks and keep spans after INTERVIEWEE_LABEL: up to next label/Q/end."""
    if not text: return ""
    txt = normalize_text(text)
    iname_up = (interviewee_name or "").strip().upper()
    ilast = iname_up.split()[-1] if iname_up else ""

    q_block = re.compile(r"(?:^|\s)Q[\.:].*?(?=(?:\s[A-Z][A-Z \.'\-]{1,40}:)|(?:\sQ[\.:])|$)",
                         flags=re.IGNORECASE | re.DOTALL)
    mod_block = re.compile(r"(?:^|\s)(?:THE\s+MODERATOR|MODERATOR)[\.:].*?(?=(?:\s[A-Z][A-Z \.'\-]{1,40}:)|(?:\sQ[\.:])|$)",
                           flags=re.IGNORECASE | re.DOTALL)
    txt = re.sub(q_block, " ", txt)
    txt = re.sub(mod_block, " ", txt)

    if not iname_up:
        return txt

    alts = [re.escape(iname_up)]
    if ilast: alts.append(re.escape(ilast))
    label_alt = "|".join(alts)

    iv_pat = re.compile(
        rf"(?:^|\s)(?:{label_alt})\s*:\s*(.*?)(?=(?:\s[A-Z][A-Z \.'\-]{{1,40}}:)|(?:\sQ[\.:])|$)",
        flags=re.IGNORECASE | re.DOTALL
    )
    pieces = [m.group(1).strip() for m in iv_pat.finditer(txt)]
    iv_text = " ".join(pieces).strip()
    return iv_text if iv_text else txt

def _normalize_sentence(s: str) -> str:
    return re.sub(r"\s+", " ", (s or "").strip()).lower()

def dedupe_repetitions(text: str) -> str:
    """Remove repeated paragraphs & sentences; collapse doubled content."""
    if not text: return ""
    paras = [p.strip() for p in re.split(r"\n{2,}|\r{2,}", text) if p.strip()]
    seen_p, keep_p = set(), []
    for p in paras:
        key = _normalize_sentence(p)
        if key in seen_p: continue
        seen_p.add(key); keep_p.append(p)
    text1 = "\n\n".join(keep_p)
    sents = [s.strip() for s in re.split(r'(?<=[\.\?\!])\s+', text1) if s.strip()]
    seen_s, keep_s = set(), []
    for s in sents:
        key = _normalize_sentence(s)
        if key in seen_s: continue
        seen_s.add(key); keep_s.append(s)
    return " ".join(keep_s)

def word_count_clean(text: str) -> int:
    if not text: return 0
    return len(re.findall(r"\b\w+\b", text))

# ---------- 1) Load interviews (source of clean words) ----------
iv = pd.read_excel(INTERVIEWS_IN, dtype=str)
for c in ("year","Team","date","interviewee","transcript"):
    if c not in iv.columns: iv[c] = ""

iv["year"] = pd.to_numeric(iv["year"], errors="coerce").astype("Int64")
iv = iv[iv["year"].notna()].copy()
iv["year"] = iv["year"].astype(int)
iv["date"] = pd.to_datetime(iv["date"], errors="coerce")
iv["interviewee"] = iv["interviewee"].fillna("")
iv["transcript"]  = iv["transcript"].fillna("").map(normalize_text)

# ---------- 2) Compute clean word counts per interview ----------
def compute_clean_wc(row):
    ans = strip_questions_and_keep_interviewee(row["transcript"], row["interviewee"])
    ans = dedupe_repetitions(ans)
    return word_count_clean(ans)

wc_key = iv[["year","date","Team","interviewee","transcript"]].copy()
wc_key["word_count_clean"] = wc_key.apply(compute_clean_wc, axis=1)

# ---------- 3) Aggregate to team-date ----------
wc_teamdate = (
    wc_key.groupby(["year","date","Team"], as_index=False)
          .agg(total_word_count_clean=("word_count_clean","sum"),
               n_interviews_clean=("interviewee","count"))
)

# ---------- 4) Load your existing team file and merge clean words ----------
if os.path.exists(TEAM_DATE_IN):
    team_dt = pd.read_csv(TEAM_DATE_IN, dtype=str)
else:
    TEAM_DATE_IN = "/content/team_date_teammate_mentions_with_series.xlsx"
    team_dt = pd.read_excel(TEAM_DATE_IN, dtype=str)

team_dt["year"] = pd.to_numeric(team_dt["year"], errors="coerce").astype("Int64")
team_dt["date"] = pd.to_datetime(team_dt["date"], errors="coerce")
team_dt["Team"] = team_dt["Team"].astype(str)

merged = team_dt.merge(wc_teamdate, on=["year","date","Team"], how="left")

# If you want a frequency using clean words:
if "total_teammate_mentions" in merged.columns:
    merged["mention_freq_clean"] = (
        pd.to_numeric(merged["total_teammate_mentions"], errors="coerce").fillna(0) /
        merged["total_word_count_clean"].replace({0:np.nan})
    )

# ---------- 5) Save updated team file with clean word counts ----------
merged.to_csv(TEAM_DATE_OUT_CSV, index=False, encoding="utf-8-sig")
merged.to_excel(TEAM_DATE_OUT_XLSX, index=False)

print("✅ Added clean word counts to team-date file:")
print("CSV :", TEAM_DATE_OUT_CSV)
print("XLSX:", TEAM_DATE_OUT_XLSX)

# quick peek
keep_cols = [c for c in ["year","date","Team","total_teammate_mentions","total_word_count",
                         "total_word_count_clean","mention_freq_clean","n_interviews","n_interviews_clean",
                         "is_division_series","is_championship_series","is_world_series","team_won"]
             if c in merged.columns]
print(merged[keep_cols].head(10).to_string(index=False))


✅ Added clean word counts to team-date file:
CSV : /content/team_date_teammate_mentions_with_series_cleanwords.csv
XLSX: /content/team_date_teammate_mentions_with_series_cleanwords.xlsx
 year       date Team total_teammate_mentions total_word_count  total_word_count_clean  mention_freq_clean n_interviews  n_interviews_clean is_division_series is_championship_series is_world_series team_won
 2014 2014-10-02  BAL                      26             8603                    2970            0.008754            4                   4                  1                      0               0      Win
 2014 2014-10-02  DET                      26             6007                    1980            0.013131            3                   3                  1                      0               0     Loss
 2014 2014-10-02  KCA                      50             9544                    3603            0.013877            4                   4                  1                      0            

In [ ]:
#Team stats
# === Setup ===
!pip -q install pybaseball==2.2.7 openpyxl rapidfuzz

import re, pandas as pd
from rapidfuzz import process, fuzz
from functools import lru_cache
from google.colab import files
from pybaseball import team_batting, team_pitching, schedule_and_record

print("Upload your Excel (must include 'year' and a team column named 'Team' or 'team').")
uploaded = files.upload()
xlsx_path = list(uploaded.keys())[0]

df_in = pd.read_excel(xlsx_path)
lower_cols = {c: c.strip().lower() for c in df_in.columns}
df_in.rename(columns=lower_cols, inplace=True)
team_col = "team" if "team" in df_in.columns else ("Team" if "Team" in df_in.columns else None)
if team_col is None: team_col = "team"
assert "year" in df_in.columns and team_col in df_in.columns, "Need 'year' and 'Team' (or 'team')."

orig_cols = list(df_in.columns)
df_in = df_in[df_in["year"].between(2014, 2024)]
df_in = df_in[df_in["year"] != 2020].copy()
df_in[team_col] = df_in[team_col].astype(str).str.strip().str.upper()

# === Normalize team codes ===
CANON = {"BAL":"BAL","BOS":"BOS","NYY":"NYY","TBR":"TBR","TOR":"TOR",
         "CHW":"CHW","CLE":"CLE","DET":"DET","KCR":"KCR","MIN":"MIN",
         "HOU":"HOU","LAA":"LAA","OAK":"OAK","SEA":"SEA","TEX":"TEX",
         "ATL":"ATL","MIA":"MIA","NYM":"NYM","PHI":"PHI","WSN":"WSN",
         "CHC":"CHC","CIN":"CIN","MIL":"MIL","PIT":"PIT","STL":"STL",
         "ARI":"ARI","COL":"COL","LAD":"LAD","SDP":"SDP","SFG":"SFG"}
ALT = {"WAS":"WSN","WSH":"WSN","TAM":"TBR","TB":"TBR","ANA":"LAA",
       "CWS":"CHW","CHA":"CHW","NYA":"NYY","NYN":"NYM",
       "SD":"SDP","SDN":"SDP","SF":"SFG","SFN":"SFG",
       "KC":"KCR","KCA":"KCR","CLV":"CLE","LA":"LAD","LAN":"LAD",
       "SLN":"STL"}

ALLOWED = list(CANON.values())
def normalize_code(raw):
    x = str(raw).strip().upper()
    if x in CANON: return CANON[x]
    if x in ALT: return ALT[x]
    best, score, _ = process.extractOne(x, ALLOWED, scorer=fuzz.WRatio)
    return best if score >= 80 else x

df_in["FG_CODE"] = df_in[team_col].apply(normalize_code)

# === Data fetchers ===
@lru_cache(None)
def fg_batting(year: int) -> pd.DataFrame:
    tb = team_batting(year)
    keep = {'Team','Season','R','OBP','Off','Def','WAR','wRC+'}
    tb = tb[[c for c in tb.columns if c in keep]].copy()
    tb = tb.rename(columns={'Team':'FG_CODE','Season':'year','R':'Runs Scored',
                            'WAR':'Offensive WAR','Def':'Defensive WAR','wRC+':'OPS+'})
    tb['year'] = tb['year'].astype(int)
    return tb

@lru_cache(None)
def fg_pitching(year: int) -> pd.DataFrame:
    tp = team_pitching(year)
    keep = [c for c in tp.columns if c in ['Team','Season','IP','ERA-','FIP','R','WAR']]
    tp = tp[keep].copy()
    tp.rename(columns={'Team':'FG_CODE','Season':'year','WAR':'Pitching WAR'}, inplace=True)
    tp['year'] = tp['year'].astype(int)
    tp['Runs Against'] = pd.to_numeric(tp['R'], errors='coerce').round().astype('Int64')
    tp['ERA+'] = tp['ERA-'].apply(lambda x: round(10000/x) if pd.notnull(x) and x>0 else pd.NA).astype('Int64')
    return tp[['year','FG_CODE','FIP','Runs Against','ERA+','Pitching WAR']]

def win_pct_from_schedule(code, year):
    try:
        s = schedule_and_record(code, year)
    except Exception:
        return pd.NA
    w = l = None
    if 'W' in s.columns and 'L' in s.columns:
        w = pd.to_numeric(s['W'], errors='coerce').dropna().max()
        l = pd.to_numeric(s['L'], errors='coerce').dropna().max()
    elif 'W-L' in s.columns:
        last = s['W-L'].dropna().astype(str).iloc[-1]
        m = re.match(r'(\d+)-(\d+)', last)
        if m: w, l = int(m.group(1)), int(m.group(2))
    return round(w/(w+l),3) if w and l else pd.NA

# === Merge all ===
years = sorted(df_in['year'].unique())
fg_bat_all = pd.concat([fg_batting(y) for y in years], ignore_index=True)
fg_pit_all = pd.concat([fg_pitching(y) for y in years], ignore_index=True)
fg_merged = pd.merge(fg_bat_all, fg_pit_all, on=['year','FG_CODE'], how='outer')

cols = ['year','FG_CODE','OPS+','ERA+','FIP','Runs Scored','Runs Against',
        'OBP','Offensive WAR','Pitching WAR','Defensive WAR']
fg_merged = fg_merged[cols]

out = df_in.merge(fg_merged, on=['year','FG_CODE'], how='left')
# Win%
win_cache = {(y, c): win_pct_from_schedule(c, int(y)) for y,c in out[['year','FG_CODE']].drop_duplicates().itertuples(index=False)}
out['Win%'] = [win_cache.get((int(y), c), pd.NA) for y,c in out[['year','FG_CODE']].itertuples(index=False)]

metric_cols = ['OPS+','ERA+','FIP','Runs Scored','Runs Against','Win%','OBP',
               'Offensive WAR','Pitching WAR','Defensive WAR']
out = out[orig_cols + [c for c in metric_cols if c not in orig_cols]]

# === Save result ===
out_path = "mlb_enriched_from_your_sheet.xlsx"
with pd.ExcelWriter(out_path, engine="openpyxl") as w:
    out.to_excel(w, index=False, sheet_name="enriched")
    df_in[['year', team_col, 'FG_CODE']].to_excel(w, index=False, sheet_name="audit_codes")

print("✅ Done! Downloading your enriched file...")
files.download(out_path)


Upload your Excel (must include 'year' and a team column named 'Team' or 'team').


Saving team_date_teammate_mentions_with_series_cleanwords.xlsx to team_date_teammate_mentions_with_series_cleanwords.xlsx
✅ Done! Downloading your enriched file...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Win Percentage

# --- Robust fix for Win% column ---
from pybaseball import team_pitching
from rapidfuzz import process, fuzz

# 1️⃣ Rebuild FG_CODE if missing
if 'FG_CODE' not in out.columns:
    print("No FG_CODE column found — regenerating from team abbreviations...")
    CANON = {
        "BAL":"BAL","BOS":"BOS","NYY":"NYY","TBR":"TBR","TOR":"TOR",
        "CHW":"CHW","CLE":"CLE","DET":"DET","KCR":"KCR","MIN":"MIN",
        "HOU":"HOU","LAA":"LAA","OAK":"OAK","SEA":"SEA","TEX":"TEX",
        "ATL":"ATL","MIA":"MIA","NYM":"NYM","PHI":"PHI","WSN":"WSN",
        "CHC":"CHC","CIN":"CIN","MIL":"MIL","PIT":"PIT","STL":"STL",
        "ARI":"ARI","COL":"COL","LAD":"LAD","SDP":"SDP","SFG":"SFG"
    }
    ALT = {"WAS":"WSN","WSH":"WSN","TAM":"TBR","TB":"TBR","TBA":"TBR","ANA":"LAA",
           "CWS":"CHW","CHA":"CHW","CHN":"CHC","NYA":"NYY","NYN":"NYM",
           "SD":"SDP","SDN":"SDP","SF":"SFG","SFN":"SFG",
           "KC":"KCR","KCA":"KCR","CLV":"CLE","LA":"LAD","LAN":"LAD","SLN":"STL"}
    allowed = list(CANON.values())
    def normalize_code(raw):
        x = str(raw).strip().upper()
        if x in CANON: return CANON[x]
        if x in ALT: return ALT[x]
        best, score, _ = process.extractOne(x, allowed, scorer=fuzz.WRatio)
        return best if score >= 80 else x

    team_col = next((c for c in out.columns if c.lower() == 'team'), None)
    out['FG_CODE'] = out[team_col].apply(normalize_code)

# 2️⃣ Collect win/loss data from FanGraphs
@lru_cache(None)
def fg_team_records(year: int) -> pd.DataFrame:
    df = team_pitching(year)
    df = df.rename(columns={'Team': 'FG_CODE'})
    df['year'] = year

    # Locate win/loss columns safely
    win_cols = [c for c in df.columns if c.lower() in ('w', 'wins')]
    loss_cols = [c for c in df.columns if c.lower() in ('l', 'losses', 'los')]
    if not win_cols or not loss_cols:
        raise ValueError(f"No W/L columns found for {year}: {df.columns.tolist()}")

    w_col, l_col = win_cols[0], loss_cols[0]
    df['Win%'] = (df[w_col] / (df[w_col] + df[l_col])).round(3)
    return df[['year', 'FG_CODE', 'Win%']]

years = sorted(out['year'].dropna().unique().astype(int))
fg_records = pd.concat([fg_team_records(y) for y in years], ignore_index=True)

# 3️⃣ Merge Win% cleanly
out = out.drop(columns=['Win%'], errors='ignore') \
         .merge(fg_records, on=['year', 'FG_CODE'], how='left')

# 4️⃣ Save new version
out_path = "mlb_enriched_with_winpct_fixed_finalpls.xlsx"
out.to_excel(out_path, index=False)
print("✅ Win% column successfully added and saved.")
files.download(out_path)

✅ Win% column successfully added and saved.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Home Field Advantage
# === USER SETTINGS ===
INPUT_XLSX  = "mlb_enriched_with_winpct_fixed_finalpls.xlsx"   # your dataset with 'date' and 'team'
SHEET_NAME  = 0                              # or "Sheet1"
DATE_COL    = "date"
TEAM_COL    = "team"
OUTPUT_XLSX = "/content/mlb_playoffs_with_hfa.xlsx"

YEARS = [y for y in range(2014, 2025) if y != 2020]  # 2014–2024, skip 2020

# === CODE ===
import os, zipfile, pandas as pd, requests
from datetime import timedelta

ZIP_URL = "https://www.retrosheet.org/downloads/postseason.zip"
ZIP_PATH = "/content/postseason.zip"
EXTRACT_DIR = "/content/retrosheet_postseason"

# ---- helpers ----
def normalize_date_any(series: pd.Series) -> pd.Series:
    """Convert to date-only; handles YYYY-MM-DD, YYYY/MM/DD, datetimes, and Retrosheet YYYYMMDD."""
    s = series.astype(str).str.strip()
    out = pd.Series(pd.NaT, index=s.index, dtype="datetime64[ns]")
    m8 = s.str.fullmatch(r"\d{8}")
    if m8.any():
        out.loc[m8] = pd.to_datetime(s.loc[m8], format="%Y%m%d", errors="coerce")
    rest = ~m8
    if rest.any():
        out.loc[rest] = pd.to_datetime(s.loc[rest], errors="coerce")
    return out.dt.normalize()

# Map both sides to a single canonical code so they match.
# Your sample uses Retrosheet-style codes like KCA, LAN, SFN, SLN, WAS, LAA.
# The only mismatch we saw was Angels as ANA (Retrosheet) vs LAA (your sheet).
TEAM_ALIAS_TO_CANON = {
    # Nationals
    "WSH": "WAS",
    # Angels
    "ANA": "LAA",
    # (Add more if you find mismatches; examples below commented out)
    # "TB": "TBA",
    # "KC": "KCA",
    # "LAD": "LAN",
    # "SFG": "SFN",
    # "STL": "SLN",
    # "CHC": "CHN",
    # "CHW": "CHA",
}

def canon(code: str) -> str:
    c = str(code).strip().upper()
    return TEAM_ALIAS_TO_CANON.get(c, c)

# ---- download & load Retrosheet postseason ----
if not os.path.exists(ZIP_PATH):
    r = requests.get(ZIP_URL, timeout=60)
    r.raise_for_status()
    with open(ZIP_PATH, "wb") as f:
        f.write(r.content)

if not os.path.exists(EXTRACT_DIR):
    os.makedirs(EXTRACT_DIR, exist_ok=True)
    with zipfile.ZipFile(ZIP_PATH, "r") as z:
        z.extractall(EXTRACT_DIR)

gi = pd.read_csv(os.path.join(EXTRACT_DIR, "gameinfo.csv"), dtype=str)

required = ["visteam","hometeam","date","season"]
missing = [c for c in required if c not in gi.columns]
if missing:
    raise RuntimeError(f"Missing expected columns in gameinfo.csv: {missing}")

gi["season"] = pd.to_numeric(gi["season"], errors="coerce")
gi = gi[gi["season"].isin(YEARS)].copy()

# Exclude Wild Card via any available descriptor columns
wc_cols = [c for c in ["gametype","round","series","ser","sername"] if c in gi.columns]
if wc_cols:
    wc_mask = False
    for c in wc_cols:
        s = gi[c].astype(str).str.upper()
        wc_mask = wc_mask | s.str.contains("WILD", na=False) | s.str.contains(r"\bWC\b", regex=True, na=False)
    gi = gi[~wc_mask].copy()

# Normalize dates & teams (to canonical)
gi["_date_norm"] = normalize_date_any(gi["date"])
gi["_home_team_norm"] = gi["hometeam"].map(canon)
gi["_away_team_norm"] = gi["visteam"].map(canon)

home_rows = gi[["_date_norm","_home_team_norm"]].rename(columns={"_home_team_norm":"_team_norm"})
home_rows["home_field_advantage"] = 1
away_rows = gi[["_date_norm","_away_team_norm"]].rename(columns={"_away_team_norm":"_team_norm"})
away_rows["home_field_advantage"] = 0
postseason_team_dates = pd.concat([home_rows, away_rows], ignore_index=True)

# ---- load your file ----
df = pd.read_excel(INPUT_XLSX, sheet_name=SHEET_NAME)
if DATE_COL not in df.columns or TEAM_COL not in df.columns:
    raise ValueError(f"Expected columns '{DATE_COL}' and '{TEAM_COL}'. Found: {list(df.columns)}")

df["_date_norm"] = normalize_date_any(df[DATE_COL])
df["_team_norm"] = df[TEAM_COL].map(canon)

# ---- first-pass merge (exact date) ----
merged = df.merge(
    postseason_team_dates,
    on=["_date_norm","_team_norm"],
    how="left"
)

# ---- second-pass fix: SAFE ±1 day retry (unambiguous only) ----
USE_SAFE_FUZZY = True  # set False to disable fuzzy matching entirely

if USE_SAFE_FUZZY:
    # Build quick lookups to test ambiguity
    # Count how many entries exist for a (date, team) key in postseason data
    key_counts = postseason_team_dates.groupby(["_date_norm","_team_norm"]).size().rename("cnt").reset_index()

    # mark rows still missing
    needs_fix = merged["home_field_advantage"].isna()
    if needs_fix.any():
        # subset of keys that failed exact match
        fix_keys = merged.loc[needs_fix, ["_date_norm","_team_norm"]].copy()

        # Helper to try a shifted day and return a Series of results,
        # but only where the (date_shift, team) key exists exactly once.
        def try_shift(delta_days: int, label: str):
            tmp = fix_keys.copy()
            tmp["_date_norm_shift"] = tmp["_date_norm"] + pd.Timedelta(days=delta_days)

            # join to counts to ensure unambiguous match (exactly one)
            tmp = tmp.merge(
                key_counts.rename(columns={"_date_norm":"_date_norm_shift"}),
                on=["_date_norm_shift","_team_norm"],
                how="left"
            )
            tmp = tmp[tmp["cnt"] == 1]  # keep only unique keys

            # fetch the actual HFA for those keys
            res = tmp.merge(
                postseason_team_dates.rename(columns={"_date_norm":"_date_norm_shift"}),
                on=["_date_norm_shift","_team_norm"],
                how="left"
            )[["home_field_advantage"]]

            res = res.rename(columns={"home_field_advantage": f"hfa_{label}"})
            res.index = tmp.index  # align back to fix_keys index positions
            return res

        # Try -1 day and +1 day
        res_minus = try_shift(-1, "minus")
        res_plus  = try_shift(+1, "plus")

        # Combine back onto the original index for rows needing fix
        candidates = pd.DataFrame(index=fix_keys.index)
        if not res_minus.empty:
            candidates = candidates.join(res_minus, how="left")
        if not res_plus.empty:
            candidates = candidates.join(res_plus, how="left")

        # Decision rule:
        # - If exactly one of hfa_minus / hfa_plus is non-null -> use it
        # - If both non-null (ambiguous) or both null -> leave as NaN
        if not candidates.empty:
            choice = candidates.apply(
                lambda r: r["hfa_minus"]
                          if pd.notna(r.get("hfa_minus")) and pd.isna(r.get("hfa_plus"))
                          else (r["hfa_plus"] if pd.notna(r.get("hfa_plus")) and pd.isna(r.get("hfa_minus"))
                                else pd.NA),
                axis=1
            )
            merged.loc[needs_fix, "home_field_advantage"] = merged.loc[needs_fix, "home_field_advantage"].fillna(choice)

# ---- save ----
out = merged.drop(columns=["_date_norm","_team_norm"])
out.to_excel(OUTPUT_XLSX, index=False)

unmatched = int(out["home_field_advantage"].isna().sum())
total = len(out)
print(f"✅ Done! Wrote: {OUTPUT_XLSX}")
print(f"Remaining unmatched (NaN) after alias + ±1 day pass: {unmatched} / {total}")

# quick peek
cols = [c for c in [DATE_COL, TEAM_COL, "home_field_advantage"] if c in out.columns]
print(out[cols].head(25).to_string(index=False))


✅ Done! Wrote: /content/mlb_playoffs_with_hfa.xlsx
Remaining unmatched (NaN) after alias + ±1 day pass: 0 / 664
      date team  home_field_advantage
2014-10-02  BAL                     1
2014-10-02  DET                     0
2014-10-02  KCA                     0
2014-10-02  LAA                     1
2014-10-03  BAL                     1
2014-10-03  DET                     0
2014-10-03  KCA                     0
2014-10-03  LAA                     1
2014-10-03  LAN                     1
2014-10-03  SFN                     0
2014-10-03  SLN                     0
2014-10-03  WAS                     1
2014-10-04  LAN                     1
2014-10-04  SFN                     0
2014-10-04  SLN                     0
2014-10-04  WAS                     1
2014-10-05  BAL                     0
2014-10-05  DET                     1
2014-10-05  KCA                     1
2014-10-05  LAA                     0
2014-10-06  LAN                     0
2014-10-06  SFN                     1
2014-10-06  SL

In [ ]:
#Momemntum - Sept Win Percentage
# === USER SETTINGS ===
INPUT_XLSX   = "/content/mlb_playoffs_with_hfa.xlsx"     # your current file (must have 'year' and 'team')
SHEET_NAME   = 0                                          # or "Sheet1"
YEAR_COL     = "year"
TEAM_COL     = "team"
OUTPUT_XLSX  = "/content/mlb_playoffs_with_hfa_and_septwinpct.xlsx"

YEARS = [y for y in range(2014, 2025) if y != 2020]       # 2014–2024, skip 2020

# === CODE ===
import io, os, zipfile, pandas as pd, requests

GL_BASE = "https://www.retrosheet.org/gamelogs"

# --- Team code canonicalization ---
# Map common user/MLB/BBR codes -> Retrosheet gamelog codes (default mapping)
ALIAS_TO_RS_BASE = {
    # AL East
    "NYY":"NYA","NYA":"NYA","BOS":"BOS","BAL":"BAL","TOR":"TOR",
    "TBR":"TBA","TB":"TBA","TBA":"TBA",
    # AL Central
    "CHW":"CHA","CWS":"CHA","CHA":"CHA","CLE":"CLE","DET":"DET",
    "KCR":"KCA","KC":"KCA","KCA":"KCA","MIN":"MIN",
    # AL West
    "OAK":"OAK","SEA":"SEA","TEX":"TEX","HOU":"HOU",
    "LAA":"LAA","ANA":"ANA",  # handled year-aware below
    # NL East
    "ATL":"ATL","MIA":"MIA","FLA":"MIA","NYM":"NYN","NYN":"NYN","PHI":"PHI",
    "WSN":"WAS","WSH":"WAS","WAS":"WAS",
    # NL Central
    "CHC":"CHN","CHN":"CHN","CIN":"CIN","MIL":"MIL","PIT":"PIT",
    "STL":"SLN","SLN":"SLN",
    # NL West
    "ARI":"ARI","COL":"COL",
    "LAD":"LAN","LAN":"LAN",
    "SDP":"SDN","SD":"SDN","SDN":"SDN",
    "SFG":"SFN","SFN":"SFN",
}

def canon_team_year(code: str, year: int) -> str:
    """
    Canonicalize a user team code to Retrosheet gamelog code, with year-aware handling for the Angels.
    Retrosheet often uses ANA (2014–2015) and LAA later.
    """
    c = str(code).strip().upper()
    rs = ALIAS_TO_RS_BASE.get(c, c)
    if c in ("LAA", "ANA"):            # Angels special-case by season
        rs = "ANA" if int(year) <= 2015 else "LAA"
    return rs

def fetch_gamelog_year(year: int) -> pd.DataFrame:
    """
    Download & read Retrosheet GL{year}.TXT; return needed fields.
    """
    url = f"{GL_BASE}/gl{year}.zip"
    zpath = f"/content/gl{year}.zip"

    if not os.path.exists(zpath):
        r = requests.get(url, timeout=60)
        r.raise_for_status()
        with open(zpath, "wb") as f:
            f.write(r.content)

    with zipfile.ZipFile(zpath, "r") as z:
        name = None
        for n in z.namelist():
            if n.upper().endswith(f"GL{year}.TXT"):
                name = n
                break
        if name is None:
            raise RuntimeError(f"Could not find GL{year}.TXT in the zip.")
        with z.open(name) as f:
            # Gamelog columns (0-based): 0=date(YYYYMMDD), 3=visteam, 6=hometeam, 9=v_score, 10=h_score
            cols = [0,3,6,9,10]
            df = pd.read_csv(f, header=None, usecols=cols, dtype=str)
            df.columns = ["date_raw","visteam","hometeam","v_score","h_score"]

    dt = pd.to_datetime(df["date_raw"], format="%Y%m%d", errors="coerce")
    df = df.assign(
        date = dt.dt.normalize(),
        month = dt.dt.month,
        season = dt.dt.year,
        v_score = pd.to_numeric(df["v_score"], errors="coerce"),
        h_score = pd.to_numeric(df["h_score"], errors="coerce")
    )
    print(f"{year}: loaded September game logs")
    return df[["date","month","season","visteam","hometeam","v_score","h_score"]]

# --- Build September win% per (team, season) from 2014–2024 (skip 2020) ---
parts = []
for yr in YEARS:
    try:
        gl = fetch_gamelog_year(yr)
    except Exception as e:
        print(f"Failed {yr}: {e}")
        continue

    sep = gl[gl["month"] == 9].copy()

    # two team-rows per game: visitor & home
    away = sep[["season","visteam","v_score","h_score"]].copy()
    away["team_rs"] = away["visteam"]
    away["win"] = (away["v_score"] > away["h_score"]).astype(int)

    home = sep[["season","hometeam","v_score","h_score"]].copy()
    home["team_rs"] = home["hometeam"]
    home["win"] = (home["h_score"] > home["v_score"]).astype(int)

    team_rows = pd.concat([
        away.rename(columns={"visteam":"opp_rs"})[["season","team_rs","win"]],
        home.rename(columns={"hometeam":"opp_rs"})[["season","team_rs","win"]],
    ], ignore_index=True)

    agg = team_rows.groupby(["season","team_rs"], as_index=False).agg(
        sept_wins=("win","sum"),
        sept_games=("win","count")
    )
    agg["sept_win_pct"] = agg["sept_wins"] / agg["sept_games"]
    parts.append(agg[["season","team_rs","sept_win_pct"]])

sept_win = pd.concat(parts, ignore_index=True) if parts else pd.DataFrame(columns=["season","team_rs","sept_win_pct"])

# As a robustness layer, also add alias duplicates (so either code will match if needed)
alias_pairs = [
    ("ANA","LAA"), ("LAA","ANA"),
    ("WAS","WSN"), ("TBA","TBR"),
    ("KCA","KCR"),
    ("CHN","CHC"), ("CHA","CHW"),
    ("LAN","LAD"), ("SFN","SFG"),
    ("SLN","STL"), ("SDN","SDP"),
    ("NYN","NYM"),
]
exp = []
for rs, user in alias_pairs:
    tmp = sept_win[sept_win["team_rs"] == rs].copy()
    if not tmp.empty:
        tmp["team_rs"] = user
        exp.append(tmp)
if exp:
    sept_win = pd.concat([sept_win] + exp, ignore_index=True)

# --- Load your playoff file and map its (year, team) to Retrosheet codes (year-aware for Angels) ---
df = pd.read_excel(INPUT_XLSX, sheet_name=SHEET_NAME)
if YEAR_COL not in df.columns or TEAM_COL not in df.columns:
    raise ValueError(f"Expected columns '{YEAR_COL}' and '{TEAM_COL}' in your file.")

df["_season"] = pd.to_numeric(df[YEAR_COL], errors="coerce").astype("Int64")
df["_team_rs"] = df.apply(lambda r: canon_team_year(r[TEAM_COL], r[YEAR_COL]), axis=1)

# --- Merge and save ---
merged = df.merge(
    sept_win.rename(columns={"season":"_season","team_rs":"_team_rs"}),
    left_on=["_season","_team_rs"],
    right_on=["_season","_team_rs"],
    how="left"
)

out = merged.drop(columns=["_season","_team_rs"])
out.to_excel(OUTPUT_XLSX, index=False)

missing = out["sept_win_pct"].isna().sum()
print(f"✅ Done! Wrote: {OUTPUT_XLSX}")
print(f"Rows without a September win% after merge: {missing} / {len(out)}")
print(out[[YEAR_COL, TEAM_COL, "sept_win_pct"]].head(20).to_string(index=False))


2014: loaded September game logs
2015: loaded September game logs
2016: loaded September game logs
2017: loaded September game logs
2018: loaded September game logs
2019: loaded September game logs
2021: loaded September game logs
2022: loaded September game logs
2023: loaded September game logs
2024: loaded September game logs
✅ Done! Wrote: /content/mlb_playoffs_with_hfa_and_septwinpct.xlsx
Rows without a September win% after merge: 0 / 664
 year team  sept_win_pct
 2014  BAL      0.629630
 2014  DET      0.615385
 2014  KCA      0.576923
 2014  LAA      0.576923
 2014  BAL      0.629630
 2014  DET      0.615385
 2014  KCA      0.576923
 2014  LAA      0.576923
 2014  LAN      0.680000
 2014  SFN      0.520000
 2014  SLN      0.653846
 2014  WAS      0.703704
 2014  LAN      0.680000
 2014  SFN      0.520000
 2014  SLN      0.653846
 2014  WAS      0.703704
 2014  BAL      0.629630
 2014  DET      0.615385
 2014  KCA      0.576923
 2014  LAA      0.576923


In [ ]:
#Opponent Matching

# === COLAB-READY: Add opponent team from Retrosheet ===
# Works with CSV or Excel inputs. Handles multiple games per day and doubleheaders.

l
# --------------------------
# 0) USER SETTINGS
# --------------------------
FILE_PATH = '/content/mlb_playoffs_with_hfa_and_septwinpct.xlsx'  # <- change to your file path
OUT_PATH  = '/content/mlb_playoffs_with_opponents.csv'            # output CSV

# Optional explicit column names (leave as None to auto-detect)
DATE_COL = 'date'           # e.g., 'date' or 'game_date'
TEAM_COL = 'team'          # e.g., 'team_abbr' or 'team'
DH_COL   = None           # doubleheader game number column if you have it (0/1/2...). Leave None if not applicable.

# Optional filter (uncomment to restrict to postseason only)
POSTSEASON_ONLY = False   # True to restrict to postseason (Retrosheet 'gametype' includes 'post')

# --------------------------
# 1) LOAD YOUR DATAFRAME
# --------------------------
def load_df(path: str) -> pd.DataFrame:
    if not os.path.exists(path):
        raise FileNotFoundError(f"File not found at: {path}\n"
                                f"Tip: in Colab, either upload the file or mount Google Drive and update FILE_PATH.")
    # Choose reader based on extension; fall back gracefully
    ext = os.path.splitext(path)[1].lower()
    if ext in ('.xlsx', '.xls'):
        return pd.read_excel(path)
    elif ext in ('.csv', '.txt'):
        # Try UTF-8 first, then Latin-1 (handles many “weird” CSVs)
        try:
            return pd.read_csv(path)
        except UnicodeDecodeError:
            return pd.read_csv(path, encoding='latin1')
    else:
        # Try Excel then CSV
        try:
            return pd.read_excel(path)
        except Exception:
            return pd.read_csv(path, low_memory=False)

df = load_df(FILE_PATH)
print(f"Loaded dataset: {FILE_PATH}  shape={df.shape}")
# display(df.head())

# --------------------------
# 2) AUTO-DETECT CORE COLUMNS (unless explicitly provided)
# --------------------------
def pick(df, names, what):
    if what in ('date', 'team', 'doubleheader'):
        # Prefer exact matches first, then case-insensitive
        for n in names:
            if n in df.columns: return n
        # case-insensitive fallback
        lower = {c.lower(): c for c in df.columns}
        for n in names:
            if n.lower() in lower: return lower[n.lower()]
    raise ValueError(f"Could not find {what} column. Looked for: {names}\nYour columns: {list(df.columns)}")

if DATE_COL is None:
    DATE_COL = pick(df, ['date','game_date','Date','DATE','gameDate'], 'date')
if TEAM_COL is None:
    TEAM_COL = pick(df, ['team_abbr','team','Team','TEAM','tm','team_code','team_name','team_name_abbr'], 'team')

if DH_COL is None:
    for cand in ['game_number','gamenumber','number','dh','DH','doubleheader_game','game_no']:
        if cand in df.columns:
            DH_COL = cand
            break

print(f"Detected columns → DATE_COL='{DATE_COL}', TEAM_COL='{TEAM_COL}', DH_COL='{DH_COL}'")

# --------------------------
# 3) NORMALIZE DATES & TEAM LABELS
# --------------------------
df = df.copy()
df[DATE_COL] = pd.to_datetime(df[DATE_COL], errors='coerce')
if df[DATE_COL].isna().any():
    bad = df[df[DATE_COL].isna()].head(10)
    raise ValueError(f"Some dates could not be parsed in '{DATE_COL}'. Example rows:\n{bad}")

df['__date'] = df[DATE_COL].dt.date

# Map your team labels → Retrosheet team codes (3 letters like NYA, LAN, CHN)
# Extend this dictionary if your labels differ.
to_retro = {
    # AL East
    'NYY':'NYA','NY':'NYA','YANKEES':'NYA','NYY ':'NYA',
    'BOS':'BOS','RED SOX':'BOS',
    'TBR':'TBA','TB':'TBA','RAYS':'TBA',
    'TOR':'TOR','BLUE JAYS':'TOR',
    'BAL':'BAL','ORIOLES':'BAL',
    # AL Central
    'CWS':'CHA','CHW':'CHA','WHITESOX':'CHA','WHITE SOX':'CHA',
    'CLE':'CLE','GUARDIANS':'CLE','INDIANS':'CLE',
    'DET':'DET','TIGERS':'DET',
    'KCR':'KCA','KC':'KCA','ROYALS':'KCA',
    'MIN':'MIN','TWINS':'MIN',
    # AL West
    'LAA':'ANA','ANGELS':'ANA','ANA':'ANA',
    'HOU':'HOU','ASTROS':'HOU',
    'OAK':'OAK','ATHLETICS':'OAK',
    'SEA':'SEA','MARINERS':'SEA',
    'TEX':'TEX','RANGERS':'TEX',
    # NL East
    'ATL':'ATL','BRAVES':'ATL',
    'MIA':'FLO','FLA':'FLO','MARLINS':'FLO',
    'NYM':'NYN','METS':'NYN',
    'PHI':'PHI','PHILLIES':'PHI',
    # Replace the Nationals mapping in your to_retro dict with:
    'WAS':'WAS','WSH':'WAS','NATIONALS':'WAS','WASHINGTON':'WAS',

    # NL Central
    'CHC':'CHN','CUBS':'CHN',
    'CIN':'CIN','REDS':'CIN',
    'MIL':'MIL','BREWERS':'MIL',
    'PIT':'PIT','PIRATES':'PIT',
    'STL':'SLN','CARDINALS':'SLN','STL ':'SLN',
    # NL West
    'ARI':'ARI','ARZ':'ARI','D-BACKS':'ARI','DBACKS':'ARI','DIAMONDBACKS':'ARI',
    'COL':'COL','ROCKIES':'COL',
    'LAD':'LAN','DODGERS':'LAN',
    'SDP':'SDN','SD':'SDN','PADRES':'SDN',
    'SFG':'SFN','SF':'SFN','GIANTS':'SFN',
}
def norm_team_to_retro(x):
    if pd.isna(x): return np.nan
    s = str(x).upper().strip()
    return to_retro.get(s, s)

df['__team_retro'] = df[TEAM_COL].apply(norm_team_to_retro)

# --------------------------
# 4) DOWNLOAD RETROSHEET GAMEINFO (OFFICIAL CSV ZIP)
# --------------------------
zip_url = "https://retrosheet.org/downloads/gameinfo.zip"
print("Downloading Retrosheet gameinfo.zip ...")
zbytes = requests.get(zip_url).content
with zipfile.ZipFile(io.BytesIO(zbytes)) as zf:
    with zf.open('gameinfo.csv') as f:
        gi = pd.read_csv(f, low_memory=False)

# Parse date (YYYYMMDD) and restrict to years in your data
gi['date'] = pd.to_datetime(gi['date'].astype(str), format='%Y%m%d', errors='coerce')
gi = gi.dropna(subset=['date'])
gi['date'] = gi['date'].dt.date

# Optional: postseason-only filter
if POSTSEASON_ONLY and 'gametype' in gi.columns:
    gi = gi[gi['gametype'].str.contains('post', case=False, na=False)]

years = sorted(pd.DatetimeIndex(df['__date']).year.unique())
gi = gi[gi['date'].map(lambda d: d.year).isin(years)]
print(f"Retrosheet rows kept for years {years}: {len(gi):,}")

# --------------------------
# 5) BUILD OPPONENT LOOKUP (DATE, [DH #], TEAM → OPPONENT)
# --------------------------
# Retrosheet columns: visteam (away), hometeam (home), number (0 single / 1/2 ... DH)
need_cols = {'visteam','hometeam','date'}
missing_cols = need_cols - set(gi.columns)
if missing_cols:
    raise RuntimeError(f"Expected columns missing in Retrosheet data: {missing_cols}. "
                       f"Available: {list(gi.columns)}")

if 'number' not in gi.columns:
    gi['number'] = 0  # fallback if absent

home = gi[['date','number','hometeam','visteam']].rename(columns={'hometeam':'team','visteam':'opponent'})
away = gi[['date','number','visteam','hometeam']].rename(columns={'visteam':'team','hometeam':'opponent'})
opp_map = pd.concat([home, away], ignore_index=True)

# Normalize codes & types
opp_map['team'] = opp_map['team'].str.upper().str.strip()
opp_map['opponent'] = opp_map['opponent'].str.upper().str.strip()
opp_map['__dh'] = pd.to_numeric(opp_map['number'], errors='coerce').fillna(0).astype(int)
opp_map = opp_map.drop(columns=['number'])

# --------------------------
# 6) MERGE (DATE + TEAM [+ DH]) → OPPONENT
# --------------------------
left_keys  = ['__date','__team_retro']
right_keys = ['date','team']

# Include doubleheader key if you have it
if DH_COL is not None:
    df['__dh'] = pd.to_numeric(df[DH_COL], errors='coerce').fillna(0).astype(int)
    right_keys.append('__dh')
    left_keys.append('__dh')
else:
    # If user doesn't have DH column, try to match without it first.
    pass

# Prepare right dataframe with (date, team[, __dh], opponent)
right_cols = ['date','team','opponent'] + ([ '__dh' ] if '__dh' in opp_map.columns else [])
opp_right = opp_map[right_cols]

merged = df.merge(
    opp_right,
    left_on=left_keys,
    right_on=right_keys,
    how='left'
)

# --------------------------
# 7) DIAGNOSTICS
# --------------------------
missing = merged[merged['opponent'].isna()]
total = len(merged)
matched = total - len(missing)
print(f"\nMatch summary: {matched:,}/{total:,} rows matched ({matched/total:.1%}).")

if not missing.empty:
    print("⚠️ Some rows did not match an opponent. Showing a sample for debugging...\n")
    # Show whatever columns exist among these
    cand_cols = [DATE_COL, TEAM_COL, '__team_retro']
    if DH_COL is not None:
        cand_cols.append(DH_COL)
    cand_cols += ['__date','__dh']  # helpers if present
    show_cols = [c for c in cand_cols if c in missing.columns]
    if show_cols:
        display(missing[show_cols].head(20))
    else:
        print("No expected columns found in unmatched frame. Available columns:")
        print(list(missing.columns))

    if '__team_retro' in missing.columns:
        print("\nUnmatched normalized team codes (top 20):")
        print(missing['__team_retro'].value_counts().head(20))

# --------------------------
# 8) FINALIZE & SAVE
# --------------------------
out = merged.rename(columns={'opponent':'opponent_abbr'})

# Clean up helper columns
drop_helpers = ['__date','__team_retro','__dh','date','team']  # 'date','team' only from right merge
out = out.drop(columns=[c for c in drop_helpers if c in out.columns])

out.to_csv(OUT_PATH, index=False)
print(f"\n✅ Done. Saved with opponent column to: {OUT_PATH}")
display(out.head(10))

# --- OPTIONAL: also save as Excel (.xlsx) ---
XLSX_PATH = OUT_PATH.replace('.csv', '.xlsx')  # same name, Excel extension
out.to_excel(XLSX_PATH, index=False)

print(f"📘 Also saved Excel version to: {XLSX_PATH}")


Loaded dataset: /content/mlb_playoffs_with_hfa_and_septwinpct.xlsx  shape=(664, 26)
Detected columns → DATE_COL='date', TEAM_COL='team', DH_COL='None'
Retrosheet rows kept for years [2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023, 2024]: 24,677

Match summary: 664/664 rows matched (100.0%).

✅ Done. Saved with opponent column to: /content/mlb_playoffs_with_opponents.csv


,year,date_x,team_x,total_teammate_mentions,total_word_count,n_interviews,team_won,is_division_series,is_championship_series,is_world_series,...,OBP,Offensive WAR,Pitching WAR,Defensive WAR,Win%,home_field_advantage,sept_win_pct,date_y,team_y,opponent_abbr
0,2014,2014-10-02,BAL,26,8603,4,Win,1,0,0,...,0.311,27.3,12.9,34.2,0.593,1,0.629630,2014-10-02,BAL,DET
1,2014,2014-10-02,DET,26,6007,3,Loss,1,0,0,...,0.331,24.4,19.7,-45.8,0.556,0,0.615385,2014-10-02,DET,BAL
2,2014,2014-10-02,KCA,50,9544,4,Win,1,0,0,...,0.314,18.6,17.2,26.6,0.549,0,0.576923,2014-10-02,KCA,ANA
3,2014,2014-10-02,LAA,39,8781,3,Loss,1,0,0,...,0.322,30.8,14.5,16.0,0.605,1,0.576923,2014-10-02,ANA,KCA
4,2014,2014-10-03,BAL,18,8503,4,Win,1,0,0,...,0.311,27.3,12.9,34.2,0.593,1,0.629630,2014-10-03,BAL,DET
5,2014,2014-10-03,DET,15,5582,3,Loss,1,0,0,...,0.331,24.4,19.7,-45.8,0.556,0,0.615385,2014-10-03,DET,BAL
6,2014,2014-10-03,KCA,19,9993,4,Win,1,0,0,...,0.314,18.6,17.2,26.6,0.549,0,0.576923,2014-10-03,KCA,ANA
7,2014,2014-10-03,LAA,28,8299,3,Loss,1,0,0,...,0.322,30.8,14.5,16.0,0.605,1,0.576923,2014-10-03,ANA,KCA
8,2014,2014-10-03,LAN,23,9464,3,Loss,1,0,0,...,0.333,32.3,19.5,24.3,0.580,1,0.680000,2014-10-03,LAN,SLN
9,2014,2014-10-03,SFN,32,9238,4,Win,1,0,0,...,0.311,25.7,9.4,55.1,0.543,0,0.520000,2014-10-03,SFN,WAS


📘 Also saved Excel version to: /content/mlb_playoffs_with_opponents.xlsx
